## NFL Prediction Model
---
* The goal of our model is to predict the outcomes of the 2019 playoffs and Super Bowl.
* We intend to later use it for a parlay gambling strategy. 

### Data
---
* We used pd.read_html to scrape data from foorballdb.com. 
---
#### Feature Exploration:
* Week Number, Home Team, Away Team, Home Score, Away Score, Outcome(W/L)

In [ ]:
!pip install -U turicreate

In [38]:
# Import Libraries
import requests
import time
import urllib.request
import pandas as pd
import tpot
import tensorflow as tf
import adanet as ad
import matplotlib.pyplot as plt

In [2]:
# Import 2012-2019 seasons
season_2019 = pd.read_html('https://www.footballdb.com/games/index.html?lg=NFL&yr=2019')
season_2018 = pd.read_html('https://www.footballdb.com/games/index.html?lg=NFL&yr=2018')
season_2017 = pd.read_html('https://www.footballdb.com/games/index.html?lg=NFL&yr=2017')
season_2016 = pd.read_html('https://www.footballdb.com/games/index.html?lg=NFL&yr=2016')
season_2015 = pd.read_html('https://www.footballdb.com/games/index.html?lg=NFL&yr=2015')
season_2014 = pd.read_html('https://www.footballdb.com/games/index.html?lg=NFL&yr=2014')
season_2013 = pd.read_html('https://www.footballdb.com/games/index.html?lg=NFL&yr=2013')
season_2012 = pd.read_html('https://www.footballdb.com/games/index.html?lg=NFL&yr=2012')

In [14]:
list_name = ['Mike', 'Chris', 'Thomas']
grades= [100, 20, 60]

for name, grade in zip(list_name, grades):
    print(name, grade)

Mike 100
Chris 20
Thomas 60


In [74]:
list_years =[season_2012, season_2013, season_2014, season_2015, season_2016, season_2017, season_2018, season_2019]
year = ['2012', '2013', '2014', '2015', '2016', '2017', '2018','2019']
cleaned_df = []
for yr, df in zip(year, list_years):
    for idx, weeks in enumerate(df):
        df_temp = weeks.drop(columns=['Box', 'Date']).rename(columns={'Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
        df_temp['wk_year'] = 'week_'+str(idx+1)
        df_temp['year'] = yr
        cleaned_df.append(df_temp)
                                              

In [75]:
len(cleaned_df)

168

In [76]:
for i in range(1, len(cleaned_df), 10):
    print(i)

1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161


In [77]:
final_df = cleaned_df[0]
for i in range(1,len(cleaned_df)):
    final_df = final_df.append(cleaned_df[i], ignore_index=True)
    

In [78]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2136 entries, 0 to 2135
Data columns (total 7 columns):
Visitor          2136 non-null object
Visitor_Score    2136 non-null int64
Home             2136 non-null object
Home_Score       2136 non-null int64
OT               126 non-null object
wk_year          2136 non-null object
year             2136 non-null object
dtypes: int64(2), object(5)
memory usage: 116.9+ KB


In [79]:
final_df['year'].value_counts()

2017    267
2019    267
2015    267
2012    267
2016    267
2013    267
2018    267
2014    267
Name: year, dtype: int64

In [80]:
pwd

'C:\\Users\\Christian\\Desktop\\python-homework'

In [81]:
final_df.head()

,Visitor,Visitor_Score,Home,Home_Score,OT,wk_year,year
0,Dallas CowboysDAL,24,New York GiantsNYG,17,NaN,week_1,2012
1,Washington RedskinsWAS,40,New Orleans SaintsNO,32,NaN,week_1,2012
2,St. Louis RamsSTL,23,Detroit LionsDET,27,NaN,week_1,2012
3,New England PatriotsNE,34,Tennessee TitansTEN,13,NaN,week_1,2012
4,Miami DolphinsMIA,10,Houston TexansHOU,30,NaN,week_1,2012


In [64]:
balance = 90
budget = 100
'Poor' if balance > budget else 'Rich'

'Rich'

In [65]:
balance > budget

False

In [66]:
#final_df['Outcome'] = final_df.apply(lambda row: 'Lost' if row.Visitor_Score > row.Home_Score else ('Tie' if  row.Visitor_Score == row.Home_Score else 'Won'), axis=1)


In [83]:
final_df['Outcome'] = final_df.apply(lambda row: 'Lost' if row.Visitor_Score > row.Home_Score else 'Won', axis=1)

In [84]:
final_df['Outcome'].value_counts()

Won     1221
Lost     915
Name: Outcome, dtype: int64

In [87]:
final_df['OT'].value_counts()

OT    126
Name: OT, dtype: int64

In [88]:
final_df.drop(columns=['OT'], inplace=True)

In [89]:
final_df.to_csv('myentiredata.csv')

In [90]:
final_df.iloc[:2125].to_csv('train.csv')

In [91]:
final_df.iloc[2125:].to_csv('test.csv')

In [93]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2136 entries, 0 to 2135
Data columns (total 7 columns):
Visitor          2136 non-null object
Visitor_Score    2136 non-null int64
Home             2136 non-null object
Home_Score       2136 non-null int64
wk_year          2136 non-null object
year             2136 non-null object
Outcome          2136 non-null object
dtypes: int64(2), object(5)
memory usage: 116.9+ KB


In [92]:
final_df.iloc[2125:]

,Visitor,Visitor_Score,Home,Home_Score,wk_year,year,Outcome
2125,Buffalo BillsBUF,19,Houston TexansHOU,22,week_18,2019,Won
2126,Tennessee TitansTEN,20,New England PatriotsNE,13,week_18,2019,Lost
2127,Minnesota VikingsMIN,26,New Orleans SaintsNO,20,week_18,2019,Lost
2128,Seattle SeahawksSEA,17,Philadelphia EaglesPHI,9,week_18,2019,Lost
2129,Minnesota VikingsMIN,10,San Francisco 49ersSF,27,week_19,2019,Won
2130,Tennessee TitansTEN,28,Baltimore RavensBAL,12,week_19,2019,Lost
2131,Houston TexansHOU,31,Kansas City ChiefsKC,51,week_19,2019,Won
2132,Seattle SeahawksSEA,23,Green Bay PackersGB,28,week_19,2019,Won
2133,Tennessee TitansTEN,24,Kansas City ChiefsKC,35,week_20,2019,Won
2134,Green Bay PackersGB,20,San Francisco 49ersSF,37,week_20,2019,Won


In [95]:
pd.get_dummies(final_df).shape

(2136, 101)

In [4]:
# Break seasons down by season and week
week_1_2019 = season_2019[0].drop(columns=['Box']).rename(columns={'Date':'Week 1-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_2_2019 = season_2019[1].drop(columns=['Box']).rename(columns={'Date':'Week 2-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_3_2019 = season_2019[2].drop(columns=['Box']).rename(columns={'Date':'Week 3-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_4_2019 = season_2019[3].drop(columns=['Box']).rename(columns={'Date':'Week 4-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_5_2019 = season_2019[4].drop(columns=['Box']).rename(columns={'Date':'Week 5-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_6_2019 = season_2019[5].drop(columns=['Box']).rename(columns={'Date':'Week 6-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_7_2019 = season_2019[6].drop(columns=['Box']).rename(columns={'Date':'Week 7-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_8_2019 = season_2019[7].drop(columns=['Box']).rename(columns={'Date':'Week 8-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_9_2019 = season_2019[8].drop(columns=['Box']).rename(columns={'Date':'Week 9-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_10_2019 = season_2019[9].drop(columns=['Box']).rename(columns={'Date':'Week 10-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_11_2019 = season_2019[10].drop(columns=['Box']).rename(columns={'Date':'Week 11-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_12_2019 = season_2019[11].drop(columns=['Box']).rename(columns={'Date':'Week 12-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_13_2019 = season_2019[12].drop(columns=['Box']).rename(columns={'Date':'Week 13-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_14_2019 = season_2019[13].drop(columns=['Box']).rename(columns={'Date':'Week 14-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_15_2019 = season_2019[14].drop(columns=['Box']).rename(columns={'Date':'Week 15-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_16_2019 = season_2019[15].drop(columns=['Box']).rename(columns={'Date':'Week 16-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_17_2019 = season_2019[16].drop(columns=['Box']).rename(columns={'Date':'Week 17-2019','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})

season_2019 = (week_1_2019,
               week_2_2019,
               week_3_2019,
               week_4_2019,
               week_5_2019,
               week_6_2019,
               week_7_2019,
               week_8_2019,
               week_9_2019,
               week_10_2019,
               week_11_2019,
               week_12_2019,
               week_13_2019,
               week_14_2019,
               week_15_2019,
               week_16_2019,
               week_17_2019)

week_1_2018 = season_2018[0].drop(columns=['Box']).rename(columns={'Date':'Week 1-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_2_2018 = season_2018[1].drop(columns=['Box']).rename(columns={'Date':'Week 2-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_3_2018 = season_2018[2].drop(columns=['Box']).rename(columns={'Date':'Week 3-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_4_2018 = season_2018[3].drop(columns=['Box']).rename(columns={'Date':'Week 4-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_5_2018 = season_2018[4].drop(columns=['Box']).rename(columns={'Date':'Week 5-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_6_2018 = season_2018[5].drop(columns=['Box']).rename(columns={'Date':'Week 6-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_7_2018 = season_2018[6].drop(columns=['Box']).rename(columns={'Date':'Week 7-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_8_2018 = season_2018[7].drop(columns=['Box']).rename(columns={'Date':'Week 8-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_9_2018 = season_2018[8].drop(columns=['Box']).rename(columns={'Date':'Week 9-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_10_2018 = season_2018[9].drop(columns=['Box']).rename(columns={'Date':'Week 10-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_11_2018 = season_2018[10].drop(columns=['Box']).rename(columns={'Date':'Week 11-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_12_2018 = season_2018[11].drop(columns=['Box']).rename(columns={'Date':'Week 12-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_13_2018 = season_2018[12].drop(columns=['Box']).rename(columns={'Date':'Week 13-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_14_2018 = season_2018[13].drop(columns=['Box']).rename(columns={'Date':'Week 14-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_15_2018 = season_2018[14].drop(columns=['Box']).rename(columns={'Date':'Week 15-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_16_2018 = season_2018[15].drop(columns=['Box']).rename(columns={'Date':'Week 16-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_17_2018 = season_2018[16].drop(columns=['Box']).rename(columns={'Date':'Week 17-2018','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})

season_2018 = (week_1_2018,
               week_2_2018,
               week_3_2018,
               week_4_2018,
               week_5_2018,
               week_6_2018,
               week_7_2018,
               week_8_2018,
               week_9_2018,
               week_10_2018,
               week_11_2018,
               week_12_2018,
               week_13_2018,
               week_14_2018,
               week_15_2018,
               week_16_2018,
               week_17_2018)

week_1_2017 = season_2017[0].drop(columns=['Box']).rename(columns={'Date':'Week 1-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_2_2017 = season_2017[1].drop(columns=['Box']).rename(columns={'Date':'Week 2-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_3_2017 = season_2017[2].drop(columns=['Box']).rename(columns={'Date':'Week 3-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_4_2017 = season_2017[3].drop(columns=['Box']).rename(columns={'Date':'Week 4-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_5_2017 = season_2017[4].drop(columns=['Box']).rename(columns={'Date':'Week 5-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_6_2017 = season_2017[5].drop(columns=['Box']).rename(columns={'Date':'Week 6-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_7_2017 = season_2017[6].drop(columns=['Box']).rename(columns={'Date':'Week 7-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_8_2017 = season_2017[7].drop(columns=['Box']).rename(columns={'Date':'Week 8-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_9_2017 = season_2017[8].drop(columns=['Box']).rename(columns={'Date':'Week 9-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_10_2017 = season_2017[9].drop(columns=['Box']).rename(columns={'Date':'Week 10-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_11_2017 = season_2017[10].drop(columns=['Box']).rename(columns={'Date':'Week 11-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_12_2017 = season_2017[11].drop(columns=['Box']).rename(columns={'Date':'Week 12-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_13_2017 = season_2017[12].drop(columns=['Box']).rename(columns={'Date':'Week 13-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_14_2017 = season_2017[13].drop(columns=['Box']).rename(columns={'Date':'Week 14-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_15_2017 = season_2017[14].drop(columns=['Box']).rename(columns={'Date':'Week 15-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_16_2017 = season_2017[15].drop(columns=['Box']).rename(columns={'Date':'Week 16-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_17_2017 = season_2017[16].drop(columns=['Box']).rename(columns={'Date':'Week 17-2017','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})

season_2017 = (week_1_2017,
               week_2_2017,
               week_3_2017,
               week_4_2017,
               week_5_2017,
               week_6_2017,
               week_7_2017,
               week_8_2017,
               week_9_2017,
               week_10_2017,
               week_11_2017,
               week_12_2017,
               week_13_2017,
               week_14_2017,
               week_15_2017,
               week_16_2017,
               week_17_2017)

week_1_2016 = season_2016[0].drop(columns=['Box']).rename(columns={'Date':'Week 1-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_2_2016 = season_2016[1].drop(columns=['Box']).rename(columns={'Date':'Week 2-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_3_2016 = season_2016[2].drop(columns=['Box']).rename(columns={'Date':'Week 3-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_4_2016 = season_2016[3].drop(columns=['Box']).rename(columns={'Date':'Week 4-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_5_2016 = season_2016[4].drop(columns=['Box']).rename(columns={'Date':'Week 5-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_6_2016 = season_2016[5].drop(columns=['Box']).rename(columns={'Date':'Week 6-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_7_2016 = season_2016[6].drop(columns=['Box']).rename(columns={'Date':'Week 7-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_8_2016 = season_2016[7].drop(columns=['Box']).rename(columns={'Date':'Week 8-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_9_2016 = season_2016[8].drop(columns=['Box']).rename(columns={'Date':'Week 9-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_10_2016 = season_2016[9].drop(columns=['Box']).rename(columns={'Date':'Week 10-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_11_2016 = season_2016[10].drop(columns=['Box']).rename(columns={'Date':'Week 11-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_12_2016 = season_2016[11].drop(columns=['Box']).rename(columns={'Date':'Week 12-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_13_2016 = season_2016[12].drop(columns=['Box']).rename(columns={'Date':'Week 13-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_14_2016 = season_2016[13].drop(columns=['Box']).rename(columns={'Date':'Week 14-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_15_2016 = season_2016[14].drop(columns=['Box']).rename(columns={'Date':'Week 15-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_16_2016 = season_2016[15].drop(columns=['Box']).rename(columns={'Date':'Week 16-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_17_2016 = season_2016[16].drop(columns=['Box']).rename(columns={'Date':'Week 17-2016','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})

season_2016 = (week_1_2016,
               week_2_2016,
               week_3_2016,
               week_4_2016,
               week_5_2016,
               week_6_2016,
               week_7_2016,
               week_8_2016,
               week_9_2016,
               week_10_2016,
               week_11_2016,
               week_12_2016,
               week_13_2016,
               week_14_2016,
               week_15_2016,
               week_16_2016,
               week_17_2016)

week_1_2015 = season_2015[0].drop(columns=['Box']).rename(columns={'Date':'Week 1-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_2_2015 = season_2015[1].drop(columns=['Box']).rename(columns={'Date':'Week 2-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_3_2015 = season_2015[2].drop(columns=['Box']).rename(columns={'Date':'Week 3-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_4_2015 = season_2015[3].drop(columns=['Box']).rename(columns={'Date':'Week 4-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_5_2015 = season_2015[4].drop(columns=['Box']).rename(columns={'Date':'Week 5-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_6_2015 = season_2015[5].drop(columns=['Box']).rename(columns={'Date':'Week 6-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_7_2015 = season_2015[6].drop(columns=['Box']).rename(columns={'Date':'Week 7-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_8_2015 = season_2015[7].drop(columns=['Box']).rename(columns={'Date':'Week 8-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_9_2015 = season_2015[8].drop(columns=['Box']).rename(columns={'Date':'Week 9-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_10_2015 = season_2015[9].drop(columns=['Box']).rename(columns={'Date':'Week 10-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_11_2015 = season_2015[10].drop(columns=['Box']).rename(columns={'Date':'Week 11-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_12_2015 = season_2015[11].drop(columns=['Box']).rename(columns={'Date':'Week 12-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_13_2015 = season_2015[12].drop(columns=['Box']).rename(columns={'Date':'Week 13-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_14_2015 = season_2015[13].drop(columns=['Box']).rename(columns={'Date':'Week 14-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_15_2015 = season_2015[14].drop(columns=['Box']).rename(columns={'Date':'Week 15-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_16_2015 = season_2015[15].drop(columns=['Box']).rename(columns={'Date':'Week 16-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_17_2015 = season_2015[16].drop(columns=['Box']).rename(columns={'Date':'Week 17-2015','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})

season_2015 = (week_1_2015,
               week_2_2015,
               week_3_2015,
               week_4_2015,
               week_5_2015,
               week_6_2015,
               week_7_2015,
               week_8_2015,
               week_9_2015,
               week_10_2015,
               week_11_2015,
               week_12_2015,
               week_13_2015,
               week_14_2015,
               week_15_2015,
               week_16_2015,
               week_17_2015)

week_1_2014 = season_2014[0].drop(columns=['Box']).rename(columns={'Date':'Week 1-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_2_2014 = season_2014[1].drop(columns=['Box']).rename(columns={'Date':'Week 2-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_3_2014 = season_2014[2].drop(columns=['Box']).rename(columns={'Date':'Week 3-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_4_2014 = season_2014[3].drop(columns=['Box']).rename(columns={'Date':'Week 4-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_5_2014 = season_2014[4].drop(columns=['Box']).rename(columns={'Date':'Week 5-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_6_2014 = season_2014[5].drop(columns=['Box']).rename(columns={'Date':'Week 6-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_7_2014 = season_2014[6].drop(columns=['Box']).rename(columns={'Date':'Week 7-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_8_2014 = season_2014[7].drop(columns=['Box']).rename(columns={'Date':'Week 8-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_9_2014 = season_2014[8].drop(columns=['Box']).rename(columns={'Date':'Week 9-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_10_2014 = season_2014[9].drop(columns=['Box']).rename(columns={'Date':'Week 10-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_11_2014 = season_2014[10].drop(columns=['Box']).rename(columns={'Date':'Week 11-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_12_2014 = season_2014[11].drop(columns=['Box']).rename(columns={'Date':'Week 12-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_13_2014 = season_2014[12].drop(columns=['Box']).rename(columns={'Date':'Week 13-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_14_2014 = season_2014[13].drop(columns=['Box']).rename(columns={'Date':'Week 14-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_15_2014 = season_2014[14].drop(columns=['Box']).rename(columns={'Date':'Week 15-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_16_2014 = season_2014[15].drop(columns=['Box']).rename(columns={'Date':'Week 16-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})
week_17_2014 = season_2014[16].drop(columns=['Box']).rename(columns={'Date':'Week 17-2014','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home Score', 'Unnamed: 5':'OT'})

season_2014 = (week_1_2014,
               week_2_2014,
               week_3_2014,
               week_4_2014,
               week_5_2014,
               week_6_2014,
               week_7_2014,
               week_8_2014,
               week_9_2014,
               week_10_2014,
               week_11_2014,
               week_12_2014,
               week_13_2014,
               week_14_2014,
               week_15_2014,
               week_16_2014,
               week_17_2014)

week_1_2013 = season_2013[0].drop(columns=['Box']).rename(columns={'Date':'Week 1-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_2_2013 = season_2013[1].drop(columns=['Box']).rename(columns={'Date':'Week 2-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_3_2013 = season_2013[2].drop(columns=['Box']).rename(columns={'Date':'Week 3-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_4_2013 = season_2013[3].drop(columns=['Box']).rename(columns={'Date':'Week 4-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_5_2013 = season_2013[4].drop(columns=['Box']).rename(columns={'Date':'Week 5-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_6_2013 = season_2013[5].drop(columns=['Box']).rename(columns={'Date':'Week 6-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_7_2013 = season_2013[6].drop(columns=['Box']).rename(columns={'Date':'Week 7-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_8_2013 = season_2013[7].drop(columns=['Box']).rename(columns={'Date':'Week 8-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_9_2013 = season_2013[8].drop(columns=['Box']).rename(columns={'Date':'Week 9-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_10_2013 = season_2013[9].drop(columns=['Box']).rename(columns={'Date':'Week 10-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_11_2013 = season_2013[10].drop(columns=['Box']).rename(columns={'Date':'Week 11-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_12_2013 = season_2013[11].drop(columns=['Box']).rename(columns={'Date':'Week 12-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_13_2013 = season_2013[12].drop(columns=['Box']).rename(columns={'Date':'Week 13-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_14_2013 = season_2013[13].drop(columns=['Box']).rename(columns={'Date':'Week 14-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_15_2013 = season_2013[14].drop(columns=['Box']).rename(columns={'Date':'Week 15-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_16_2013 = season_2013[15].drop(columns=['Box']).rename(columns={'Date':'Week 16-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_17_2013 = season_2013[16].drop(columns=['Box']).rename(columns={'Date':'Week 17-2013','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})

season_2013 = (week_1_2013,
               week_2_2013,
               week_3_2013,
               week_4_2013,
               week_5_2013,
               week_6_2013,
               week_7_2013,
               week_8_2013,
               week_9_2013,
               week_10_2013,
               week_11_2013,
               week_12_2013,
               week_13_2013,
               week_14_2013,
               week_15_2013,
               week_16_2013,
               week_17_2013)

week_1_2012 = season_2012[0].drop(columns=['Box']).rename(columns={'Date':'Week 1-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_2_2012 = season_2012[1].drop(columns=['Box']).rename(columns={'Date':'Week 2-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_3_2012 = season_2012[2].drop(columns=['Box']).rename(columns={'Date':'Week 3-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_4_2012 = season_2012[3].drop(columns=['Box']).rename(columns={'Date':'Week 4-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_5_2012 = season_2012[4].drop(columns=['Box']).rename(columns={'Date':'Week 5-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_6_2012 = season_2012[5].drop(columns=['Box']).rename(columns={'Date':'Week 6-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_7_2012 = season_2012[6].drop(columns=['Box']).rename(columns={'Date':'Week 7-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'}),
week_8_2012 = season_2012[7].drop(columns=['Box']).rename(columns={'Date':'Week 8-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_9_2012 = season_2012[8].drop(columns=['Box']).rename(columns={'Date':'Week 9-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_10_2012 = season_2012[9].drop(columns=['Box']).rename(columns={'Date':'Week 10-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_11_2012 = season_2012[10].drop(columns=['Box']).rename(columns={'Date':'Week 11-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_12_2012 = season_2012[11].drop(columns=['Box']).rename(columns={'Date':'Week 12-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_13_2012 = season_2012[12].drop(columns=['Box']).rename(columns={'Date':'Week 13-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_14_2012 = season_2012[13].drop(columns=['Box']).rename(columns={'Date':'Week 14-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_15_2012 = season_2012[14].drop(columns=['Box']).rename(columns={'Date':'Week 15-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_16_2012 = season_2012[15].drop(columns=['Box']).rename(columns={'Date':'Week 16-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})
week_17_2012 = season_2012[16].drop(columns=['Box']).rename(columns={'Date':'Week 17-2012','Unnamed: 2': 'Visitor_Score', 'Unnamed: 4':'Home_Score', 'Unnamed: 5':'OT'})

season_2012 = (week_1_2012,
               week_2_2012,
               week_3_2012,
               week_4_2012,
               week_5_2012,
               week_6_2012,
               week_7_2012,
               week_8_2012,
               week_9_2012,
               week_10_2012,
               week_11_2012,
               week_12_2012,
               week_13_2012,
               week_14_2012,
               week_15_2012,
               week_16_2012,
               week_17_2012)

In [ ]:
season_2012

In [ ]:
!pip install -U turicreate

In [ ]:
pandas.to_csv

In [5]:
week_17_2012


,Week 17-2012,Visitor,Visitor_Score,Home,Home_Score,OT
0,12/30/201212/30,Houston TexansHOU,16,Indianapolis ColtsIND,28,NaN
1,12/30/201212/30,Tampa Bay BuccaneersTB,22,Atlanta FalconsATL,17,NaN
2,12/30/201212/30,Cleveland BrownsCLE,10,Pittsburgh SteelersPIT,24,NaN
3,12/30/201212/30,Baltimore RavensBAL,17,Cincinnati BengalsCIN,23,NaN
4,12/30/201212/30,Chicago BearsCHI,26,Detroit LionsDET,24,NaN
5,12/30/201212/30,New York JetsNYJ,9,Buffalo BillsBUF,28,NaN
6,12/30/201212/30,Philadelphia EaglesPHI,7,New York GiantsNYG,42,NaN
7,12/30/201212/30,Jacksonville JaguarsJAX,20,Tennessee TitansTEN,38,NaN
8,12/30/201212/30,Carolina PanthersCAR,44,New Orleans SaintsNO,38,NaN
9,12/30/201212/30,Green Bay PackersGB,34,Minnesota VikingsMIN,37,NaN


In [27]:
week_17_2012['OT'].value_counts()

Series([], Name: OT, dtype: int64)

In [16]:
week_17_2012['Outcome'] = week_17_2012.apply(lambda row: row.Visitor_Score > row.Home_Score, axis=1)
week_17_2012

,Week 17-2012,Visitor,Visitor_Score,Home,Home_Score,OT,Outcome
0,12/30/201212/30,Houston TexansHOU,16,Indianapolis ColtsIND,28,NaN,False
1,12/30/201212/30,Tampa Bay BuccaneersTB,22,Atlanta FalconsATL,17,NaN,True
2,12/30/201212/30,Cleveland BrownsCLE,10,Pittsburgh SteelersPIT,24,NaN,False
3,12/30/201212/30,Baltimore RavensBAL,17,Cincinnati BengalsCIN,23,NaN,False
4,12/30/201212/30,Chicago BearsCHI,26,Detroit LionsDET,24,NaN,True
5,12/30/201212/30,New York JetsNYJ,9,Buffalo BillsBUF,28,NaN,False
6,12/30/201212/30,Philadelphia EaglesPHI,7,New York GiantsNYG,42,NaN,False
7,12/30/201212/30,Jacksonville JaguarsJAX,20,Tennessee TitansTEN,38,NaN,False
8,12/30/201212/30,Carolina PanthersCAR,44,New Orleans SaintsNO,38,NaN,True
9,12/30/201212/30,Green Bay PackersGB,34,Minnesota VikingsMIN,37,NaN,False


In [23]:
Outcome = []

for outcome in week_17_2012['Visitor_Score' and 'Home_Score']:
    if 'Outcome' == False:
        Outcome.append('Home Win')
    elif 'Outcome' == True:
        Outcome.append('Visitor Win')
    else:
        Outcome.append('Tie')
        

week_17_2012

,Week 17-2012,Visitor,Visitor_Score,Home,Home_Score,OT,Outcome
0,12/30/201212/30,Houston TexansHOU,16,Indianapolis ColtsIND,28,NaN,Visitor Win
1,12/30/201212/30,Tampa Bay BuccaneersTB,22,Atlanta FalconsATL,17,NaN,Visitor Win
2,12/30/201212/30,Cleveland BrownsCLE,10,Pittsburgh SteelersPIT,24,NaN,Visitor Win
3,12/30/201212/30,Baltimore RavensBAL,17,Cincinnati BengalsCIN,23,NaN,Visitor Win
4,12/30/201212/30,Chicago BearsCHI,26,Detroit LionsDET,24,NaN,Visitor Win
5,12/30/201212/30,New York JetsNYJ,9,Buffalo BillsBUF,28,NaN,Visitor Win
6,12/30/201212/30,Philadelphia EaglesPHI,7,New York GiantsNYG,42,NaN,Visitor Win
7,12/30/201212/30,Jacksonville JaguarsJAX,20,Tennessee TitansTEN,38,NaN,Visitor Win
8,12/30/201212/30,Carolina PanthersCAR,44,New Orleans SaintsNO,38,NaN,Visitor Win
9,12/30/201212/30,Green Bay PackersGB,34,Minnesota VikingsMIN,37,NaN,Visitor Win


In [20]:
if week_17_2012.loc[0]['Visitor_Score'] > week_17_2012.loc[0]['Home_Score']:
    print('Visitors Won!')
else:
    print('Visitors Lose')
    

Visitors Lose


In [21]:
outcome = []
for i in week_17_2012['Visitor_Score' and 'Home_Score']:
    if 'Visitor Score' > 'Home Score':
        outcome.append('Visitor Win')
    elif 'Visitor Score' < 'Home Score':
        outcome.append('Home Win')
    else:
        outcome.append('Tie')
        
week_17_2012['Outcome'] = outcome

In [22]:
week_17_2012

,Week 17-2012,Visitor,Visitor_Score,Home,Home_Score,OT,Outcome
0,12/30/201212/30,Houston TexansHOU,16,Indianapolis ColtsIND,28,NaN,Visitor Win
1,12/30/201212/30,Tampa Bay BuccaneersTB,22,Atlanta FalconsATL,17,NaN,Visitor Win
2,12/30/201212/30,Cleveland BrownsCLE,10,Pittsburgh SteelersPIT,24,NaN,Visitor Win
3,12/30/201212/30,Baltimore RavensBAL,17,Cincinnati BengalsCIN,23,NaN,Visitor Win
4,12/30/201212/30,Chicago BearsCHI,26,Detroit LionsDET,24,NaN,Visitor Win
5,12/30/201212/30,New York JetsNYJ,9,Buffalo BillsBUF,28,NaN,Visitor Win
6,12/30/201212/30,Philadelphia EaglesPHI,7,New York GiantsNYG,42,NaN,Visitor Win
7,12/30/201212/30,Jacksonville JaguarsJAX,20,Tennessee TitansTEN,38,NaN,Visitor Win
8,12/30/201212/30,Carolina PanthersCAR,44,New Orleans SaintsNO,38,NaN,Visitor Win
9,12/30/201212/30,Green Bay PackersGB,34,Minnesota VikingsMIN,37,NaN,Visitor Win


In [33]:
week_17_2012['Visitor Score'] > week_17_2012['Home Score']

0     False
1      True
2     False
3     False
4      True
5     False
6     False
7     False
8      True
9     False
10    False
11    False
12    False
13    False
14    False
15    False
dtype: bool